In [ ]:
from pathlib import Path
import pickle
import pandas as pd

project_folder = Path.home() / 'Documents' / 'TFG'
data_folder = project_folder / 'data'
images_folder = data_folder / 'images'
masks_folder = data_folder / 'semantic_segmentations'

with open(data_folder / 'dataset_info.pkl', "rb") as f:
	dataset_info = pickle.load(f)

df_train = dataset_info['df_train']

print(df_train.head())

       image_id   dx
0  ISIC_0027419  bkl
1  ISIC_0025030  bkl
2  ISIC_0026769  bkl
3  ISIC_0025661  bkl
4  ISIC_0031633  bkl


In [ ]:
import torch
from torchvision import transforms
from tqdm import tqdm
from PIL import Image

# Inicializar acumuladores
mean = torch.zeros(3)
std = torch.zeros(3)
n_pixels = 0

# Transformación a tensor (valores entre 0 y 1)
to_tensor = transforms.ToTensor()

print("Calculando mean y std...")

for _, row in tqdm(df_train.iterrows(), total=len(df_train)):
	image_id = row['image_id']
	image_path = images_folder / f"{image_id}.jpg"

	try:
		image = Image.open(image_path).convert("RGB")
		tensor = to_tensor(image)  # shape: [C, H, W]
        
        # Acumulamos media y std
		n_pixels += tensor.shape[1] * tensor.shape[2]  # H * W
		mean += tensor.sum(dim=(1, 2))  # Sum over H and W
		std += (tensor ** 2).sum(dim=(1, 2))
    
	except Exception as e:
		print(f"Error con {image_id}: {e}")

# Calcular mean y std finales
mean /= n_pixels
std = (std / n_pixels - mean ** 2).sqrt()

print(f"\n✅ Media por canal: {mean}")
print(f"✅ Desviación estándar por canal: {std}")

Calculando mean y std...


100%|██████████| 8010/8010 [05:44<00:00, 23.23it/s]



✅ Media por canal: tensor([0.7565, 0.5503, 0.5762])
✅ Desviación estándar por canal: tensor([0.1427, 0.1534, 0.1717])


In [ ]:
import torch
from torchvision import transforms
from tqdm import tqdm
from PIL import Image

# Inicializar acumuladores
mean = torch.zeros(3)
std = torch.zeros(3)
n_pixels = 0

# Transformación a tensor (valores entre 0 y 1)
to_tensor = transforms.ToTensor()

print("Calculando mean y std...")

for _, row in tqdm(df_train.iterrows(), total=len(df_train)):
	image_id = row['image_id']
	image_path = images_folder / f"{image_id}.jpg"

	try:
		image = Image.open(image_path).convert("RGB")
		tensor = to_tensor(image)  # shape: [C, H, W]
        
        # Acumulamos media y std
		n_pixels += tensor.shape[1] * tensor.shape[2]  # H * W
		mean += tensor.sum(dim=(1, 2))  # Sum over H and W
		std += (tensor ** 2).sum(dim=(1, 2))
    
	except Exception as e:
		print(f"Error con {image_id}: {e}")

# Calcular mean y std finales
std = ((std - (mean ** 2 / n_pixels)) / (n_pixels - 1)).sqrt()
mean /= n_pixels


print(f"\n✅ Media por canal: {mean}")
print(f"✅ Desviación estándar por canal: {std}")

Calculando mean y std...


  0%|          | 0/9010 [00:00<?, ?it/s]

100%|██████████| 9010/9010 [03:40<00:00, 40.95it/s]


✅ Media por canal: tensor([0.7604, 0.5478, 0.5729])
✅ Desviación estándar por canal: tensor([0.1420, 0.1531, 0.1710])
